# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install tensorflow        

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2022-04-25 14:05:13.561701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 14:05:13.561748: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/home/neosoft/Downloads/mask_dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 773 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/home/neosoft/Downloads/mask_dataset/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 127 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

2022-04-25 14:05:17.216830: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-25 14:05:17.216890: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-25 14:05:17.216922: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (neosoft-Latitude-E7270): /proc/driver/nvidia/version does not exist
2022-04-25 14:05:17.217713: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
25/25 [==============================] - 8s 276ms/step - loss: 0.4356 - accuracy: 0.8176 - val_loss: 0.4898 - val_accuracy: 0.7638
Epoch 2/25
25/25 [==============================] - 6s 254ms/step - loss: 0.2997 - accuracy: 0.8680 - val_loss: 0.4984 - val_accuracy: 0.7795
Epoch 3/25
25/25 [==============================] - 6s 231ms/step - loss: 0.2432 - accuracy: 0.8913 - val_loss: 0.4594 - val_accuracy: 0.8189
Epoch 4/25
25/25 [==============================] - 6s 231ms/step - loss: 0.2009 - accuracy: 0.9107 - val_loss: 0.2989 - val_accuracy: 0.8661
Epoch 5/25
25/25 [==============================] - 6s 222ms/step - loss: 0.1709 - accuracy: 0.9224 - val_loss: 0.5583 - val_accuracy: 0.8031
Epoch 6/25
25/25 [==============================] - 6s 233ms/step - loss: 0.1603 - accuracy: 0.9263 - val_loss: 0.2710 - val_accuracy: 0.8898
Epoch 7/25
25/25 [==============================] - 6s 231ms/step - loss: 0.1360 - accuracy: 0.9418 - val_loss: 0.2828 - val_accuracy: 0.8976
Epoch 

## Part 4 - Making a single prediction

In [30]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/home/neosoft/Downloads/mask_dataset/image_2022_04_22T14_11_29_438Z.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'Properly worn mask'
else:
  prediction = 'Improperly worn mask'

In [31]:
print(prediction)

Improperly worn mask
